https://github.com/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/llama_index/llama_index_async.ipynb

能跑通，但是没有评估数据？

In [1]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.chunk_size = 128
Settings.chunk_overlap = 16

Settings.llm = OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 27.1 ms, sys: 0 ns, total: 27.1 ms
Wall time: 26.5 ms


In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(top_k=1)

In [4]:
response = query_engine.aquery("What did the author do growing up?")

print(response)  # should be awaitable
print(await response)

<coroutine object Dispatcher.span.<locals>.async_wrapper at 0x7f98fabda180>
The author was intimately involved with building the infrastructure of the web for most of their time.


In [5]:
import os
from trulens_eval.feedback.provider import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"
os.environ["OPENAI_API_BASE"] = "https://ape:3000/v1"
provider = OpenAI(
    model_engine="qwen2"
)

In [6]:
from trulens_eval import Feedback

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    provider.relevance, name="QA Relevance"
).on_input_output()

✅ In QA Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In QA Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [7]:
from trulens_eval import TruLlama

tru_query_engine_recorder = TruLlama(query_engine, feedbacks=[f_qa_relevance])

In [8]:
with tru_query_engine_recorder as recording:
    response = await query_engine.aquery("What did the author do growing up?")

print(response)

The author grew up building the infrastructure of the web.


In [9]:
tru.get_records_and_feedback(app_ids=[])[
    0
] 

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,app_hash_7961a393bc068081d7696b1807287db6,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_be0b9aed790c90da0fad60952ab711b6,"""Could not determine main input from [QueryBun...","""Node ID: 51d220f5-9da7-424f-9941-fd24b92b5921...",-,"{""record_id"": ""record_hash_be0b9aed790c90da0fa...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-05T19:52:20.148522"", ""...",2024-08-05T19:52:20.350654,0,0,0.0
1,app_hash_7961a393bc068081d7696b1807287db6,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_a5569a79068e15e7ca6ed1c306fa4de5,"""Could not determine main input from [OpenAILi...","""assistant: The author grew up building the in...",-,"{""record_id"": ""record_hash_a5569a79068e15e7ca6...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-05T19:52:20.675519"", ""...",2024-08-05T19:52:21.147342,0,0,0.0
2,app_hash_7961a393bc068081d7696b1807287db6,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_988d8f7993110356253567c14fb1875f,"""Could not determine main input from [[ChatMes...","""assistant: The author grew up building the in...",-,"{""record_id"": ""record_hash_988d8f7993110356253...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-05T19:52:20.530410"", ""...",2024-08-05T19:52:21.194737,0,0,0.0
3,app_hash_7961a393bc068081d7696b1807287db6,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_7b0b9048f53a9b4489b81f83a8b57b1e,"""What did the author do growing up?""","""The author grew up building the infrastructur...",-,"{""record_id"": ""record_hash_7b0b9048f53a9b4489b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-05T19:52:17.896035"", ""...",2024-08-05T19:52:21.248099,3,0,0.0
